# コードの高速化

課題2,4で書いたDAアルゴリズムのコードの高速化をしてみる

prop_prefsとresp_prefsがVectorの時のやつを弄る。参照はここ（<https://gist.github.com/bicycle1885/626f59ff9e0375573470>）

In [1]:
function deferred_acceptance(m_prefs::Vector{Vector{Int}},f_prefs::Vector{Vector{Int}},caps::Vector{Int}) #元の関数
    m = length(m_prefs)
    n = length(f_prefs)
    prop_prefs = zeros(Int64,n+1,m)
    resp_prefs = zeros(Int64,m+1,n)
    
    for male in 1:m
        num = length(m_prefs[male])
        for p in 1:num
            prop_prefs[p,male] = m_prefs[male][p]
        end
    end
    for fem in 1:n
        numf = length(f_prefs[fem])
        for q in 1:numf
            resp_prefs[q,fem] = f_prefs[fem][q]
        end
    end

    m = size(prop_prefs,2)
    n = size(resp_prefs,2)
    prop_matches = zeros(Int64,m)
    L = sum(caps)
    resp_matches = zeros(Int64,L)
    unchanged_counter = 0
    next_m_approach = ones(Int64,m)
    
    indptr = Array{Int64}(n+1)
    indptr[1] = 1
    for i in 1:n
        indptr[i+1] = indptr[i] + caps[i]
    end
    
    while unchanged_counter < m
        unchanged_counter = 0
        for h in 1:m
            if prop_matches[h] == 0
                d = prop_prefs[next_m_approach[h],h]
                if d == 0
                    prop_matches[h] = 0
                    unchanged_counter += 1
                else
                    a=resp_matches[indptr[d]:indptr[d+1]-1]
                    b=zeros(Int64,caps[d])
                    for i in 1:caps[d]
                        b[i] = findfirst(resp_prefs[:,d],a[i])
                    end
                    c = maximum(b)
                    x = findfirst(resp_prefs[:,d],h)
                    if c > x && x != 0　
                        prop_matches[h] = d
                        r = findfirst(a,resp_prefs[c,d])
                        if resp_matches[indptr[d]-1+r] != 0
                            prop_matches[resp_prefs[c,d]] = 0
                            next_m_approach[resp_prefs[c,d]] += 1
                        end
                        resp_matches[indptr[d]-1+r] = h
                    else
                        next_m_approach[h] += 1
                    end
                end
            else unchanged_counter += 1 
            end
        end
    end
    return prop_matches,resp_matches,indptr
end

deferred_acceptance (generic function with 1 method)

In [2]:
include("matching_tools.jl")

_randperm2d!

In [3]:
prop_prefs, resp_prefs, caps = random_prefs(2000, 2000, ReturnCaps, allow_unmatched=false);

In [4]:
function mat2vecs{T<:Integer}(prefs::Matrix{T})
    return [prefs[1:findfirst(prefs[:, j], 0)-1, j] for j in 1:size(prefs, 2)]
end

mat2vecs (generic function with 1 method)

In [5]:
prop_prefs_v = mat2vecs(prop_prefs)
resp_prefs_v = mat2vecs(resp_prefs);

In [6]:
@time deferred_acceptance(prop_prefs_v,resp_prefs_v,caps)
@time deferred_acceptance(prop_prefs_v,resp_prefs_v,caps)
@time deferred_acceptance(prop_prefs_v,resp_prefs_v,caps)

 21.432121 seconds (5.55 M allocations: 30.579 GiB, 10.41% gc time)
 17.680196 seconds (5.52 M allocations: 30.578 GiB, 8.56% gc time)
 17.570870 seconds (5.52 M allocations: 30.578 GiB, 8.01% gc time)


([1350, 1997, 1028, 1814, 1069, 991, 884, 1226, 1986, 1668  …  1126, 534, 1580, 7, 752, 1701, 385, 1147, 1790, 782], [129, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 994, 1499, 3337, 3411, 4067, 4445, 5268, 5749, 6790  …  1979201, 1981040, 1982795, 1983720, 1984910, 1986856, 1988255, 1989857, 1990389, 1991055])

JuliaのProfileモジュールを使用してプロファイリングを行う。

In [7]:
Profile.clear()
@profile deferred_acceptance(prop_prefs_v,resp_prefs_v,caps)
Profile.print()

4    .\array.jl:1213; findnext(::Array{Int64,1}, ::Int...
6636 .\task.jl:335; (::IJulia.##11#14)()
 6636 ...Julia\src\eventloop.jl:8; eventloop(::ZMQ.Socket)
  6636 ...rc\execute_request.jl:160; execute_request(::ZMQ.Socket, ::...
   6635 .\<missing>:?; anonymous
    6635 .\profile.jl:23; macro expansion
     8    .\In[1]:4; deferred_acceptance(::Array{A...
      5 .\array.jl:227; fill!(::Array{Int64,2}, ::Int64)
      1 .\array.jl:228; fill!(::Array{Int64,2}, ::Int64)
     8    .\In[1]:5; deferred_acceptance(::Array{A...
      5 .\array.jl:227; fill!(::Array{Int64,2}, ::Int64)
      2 .\array.jl:228; fill!(::Array{Int64,2}, ::Int64)
     7    .\In[1]:10; deferred_acceptance(::Array{A...
     1    .\In[1]:15; deferred_acceptance(::Array{A...
     7    .\In[1]:16; deferred_acceptance(::Array{A...
     4    .\In[1]:24; deferred_acceptance(::Array{A...
     9    .\In[1]:43; deferred_acceptance(::Array{A...
      3 .\array.jl:528; getindex
      6 .\array.jl:530; getindex
     6    .\In[1]

とりあえず一番左の数字が大きいところが時間がかかっている部分らしい。ここだと46行目のfindfirstのところらしいですが・・・

In [8]:
@code_warntype deferred_acceptance(prop_prefs_v,resp_prefs_v,caps) #変数の型を調べる

Variables:
  #self#::#deferred_acceptance
  m_prefs::Array{Array{Int64,1},1}
  f_prefs::Array{Array{Int64,1},1}
  caps::Array{Int64,1}
  p::Int64
  #temp#@_6::Int64
  male::Int64
  num::Int64
  #temp#@_9::Int64
  q::Int64
  #temp#@_11::Int64
  fem::Int64
  numf::Int64
  #temp#@_14::Int64
  i@_15::Int64
  #temp#@_16::Int64
  i@_17::Int64
  #temp#@_18::Int64
  h::Int64
  d::Int64
  a::Any
  b@_22::Array{Int64,1}
  c::Int64
  x::Int64
  r::Int64
  #temp#@_26::Int64
  prop_prefs::Array{Int64,2}
  resp_prefs::Array{Int64,2}
  m::Int64
  n::Int64
  prop_matches::Array{Int64,1}
  L::Int64
  resp_matches::Array{Int64,1}
  unchanged_counter::Int64
  next_m_approach::Array{Int64,1}
  indptr::Array{Int64,1}
  #temp#@_37::Bool
  lI::Int64
  X::Array{Int64,1}
  z@_40::Int64
  b@_41::Bool
  #temp#@_42::Int64
  z@_43::Int64
  b@_44::Bool
  #temp#@_45::Int64

Body:
  begin 
      NewvarNode(:(prop_matches::Array{Int64,1}))
      NewvarNode(:(L::Int64))
      NewvarNode(:(resp_matches::Array{Int64,1}))

Any,Unionとなっている部分があればそこを直す。今回はaがAnyになっているが、これは場合によって長さが変わるベクトルなのでAnyと表示されているようです（a::Array{Int64,1}=…としてもAnyのまま）。  
となると型の部分で直すとこはなさそうなので、アルゴリズム自体で無駄な部分を考える。今回は必要以上に反復されている46行目のfindfirst～の部分を削る。  
最初から自身の選好の順位で記録すればわざわざ何回もfindfirstする必要がないのでは？

In [9]:
function deferred_acceptance_2(m_prefs::Vector{Vector{Int}},f_prefs::Vector{Vector{Int}},caps::Vector{Int}) #弄った関数
    m = length(m_prefs)
    n = length(f_prefs)
    prop_prefs = zeros(Int64,n+1,m)
    resp_prefs = zeros(Int64,m+1,n)
    
    for male in 1:m
        num = length(m_prefs[male])
        for p in 1:num
            prop_prefs[p,male] = m_prefs[male][p]
        end
    end
    for fem in 1:n
        numf = length(f_prefs[fem])
        for q in 1:numf
            resp_prefs[q,fem] = f_prefs[fem][q]
        end
    end

    m = size(prop_prefs,2)
    n = size(resp_prefs,2)
    prop_matches = zeros(Int64,m)
    L = sum(caps)
    resp_matches = zeros(Int64,L)
        
    indptr = Array{Int64}(n+1)
    indptr[1] = 1
    for i in 1:n
        indptr[i+1] = indptr[i] + caps[i]
    end
    
    for fnum in 1:n
        zeropref = findfirst(resp_prefs[:,fnum],0)
        for wm in indptr[fnum]:indptr[fnum+1]-1
            resp_matches[wm] = zeropref
        end
    end
    
    next = 1
    next_m_approach = ones(Int64,m)


    while next == 1
        for h in 1:m
            next = 0
            if prop_matches[h] == 0
                d = prop_prefs[next_m_approach[h],h]
                if d == 0
                    prop_matches[h] = 0
                else
                    if next != 1
                        next =1
                    end
                    a = resp_matches[indptr[d]:indptr[d+1]-1]
                    c = maximum(a)
                    x = findfirst(resp_prefs[:,d],h)
                    if c > x && x != 0
                        prop_matches[h] = d
                        r = findfirst(a,c)
                        if resp_matches[indptr[d]-1+r] != findfirst(resp_prefs[:,d],0)
                            prop_matches[resp_prefs[c,d]] = 0
                            next_m_approach[resp_prefs[c,d]] += 1
                        end
                        resp_matches[indptr[d]-1+r] = x
                    else
                        next_m_approach[h] += 1
                    end
                end
            end
        end
    end
    
    for iss in 1:n
        for isss in indptr[iss]:indptr[iss+1]-1
            resp_matches[isss] = resp_prefs[resp_matches[isss],iss]
        end
    end

    return prop_matches,resp_matches,indptr
end

deferred_acceptance_2 (generic function with 1 method)

In [10]:
@time deferred_acceptance_2(prop_prefs_v,resp_prefs_v,caps)
@time deferred_acceptance_2(prop_prefs_v,resp_prefs_v,caps)
@time deferred_acceptance_2(prop_prefs_v,resp_prefs_v,caps)

  1.198959 seconds (49.32 k allocations: 186.172 MiB, 26.23% gc time)
  0.311945 seconds (18.46 k allocations: 184.781 MiB, 42.51% gc time)
  0.317626 seconds (18.46 k allocations: 184.781 MiB, 41.28% gc time)


([1350, 1997, 1028, 1814, 1069, 991, 884, 1226, 1986, 1668  …  1126, 534, 1580, 7, 752, 1701, 385, 1147, 1790, 782], [129, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 994, 1499, 3337, 3411, 4067, 4445, 5268, 5749, 6790  …  1979201, 1981040, 1982795, 1983720, 1984910, 1986856, 1988255, 1989857, 1990389, 1991055])

無事速くなりました。

In [11]:
Profile.clear()
@profile deferred_acceptance_2(prop_prefs_v,resp_prefs_v,caps)
Profile.print()

144 .\task.jl:335; (::IJulia.##11#14)()
 144 ...Julia\src\eventloop.jl:8; eventloop(::ZMQ.Socket)
  144 ...rc\execute_request.jl:160; execute_request(::ZMQ.Socket, ::...
   144 .\<missing>:?; anonymous
    144 .\profile.jl:23; macro expansion
     10 .\In[9]:4; deferred_acceptance_2(::Array{...
      7 .\array.jl:227; fill!(::Array{Int64,2}, ::Int64)
      2 .\array.jl:228; fill!(::Array{Int64,2}, ::Int64)
     9  .\In[9]:5; deferred_acceptance_2(::Array{...
      6 .\array.jl:227; fill!(::Array{Int64,2}, ::Int64)
      3 .\array.jl:228; fill!(::Array{Int64,2}, ::Int64)
     9  .\In[9]:10; deferred_acceptance_2(::Array{...
     3  .\In[9]:15; deferred_acceptance_2(::Array{...
     7  .\In[9]:16; deferred_acceptance_2(::Array{...
     4  .\In[9]:24; deferred_acceptance_2(::Array{...
     9  .\In[9]:33; deferred_acceptance_2(::Array{...
      9 .\abstractarray.jl:882; getindex
       9 .\multidimensional.jl:438; _getindex
        9 .\multidimensional.jl:442; macro expansion
         9 .\

この結果を見るとfindfirst自体が遅いのではないかという気がします（  
MyMatchingの関数も更新して一応テストを行います

In [12]:
using MyMatching

In [13]:
Pkg.test("MyMatching")

INFO: Testing MyMatching


Test Summary:               | Pass  Total
Testing deferred acceptance |   24     24


INFO: MyMatching tests passed


あとはお互い第一志望同士だったらその人のところは飛ばして反復の回数を減らす、などでしょうか